In [9]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression 
from math import sqrt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from statsmodels.formula.api import ols

import prepare_telco
from pydataset import data

#from math import sqrt
#from sklearn.metrics import mean_squared_error

### 1. Load the tips dataset.

    -Create a column named price_per_person. This should be the total bill divided by the party size.
    -Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?
    -Use select k best to select the top 2 features for predicting tip amount. What are they?
    -Use recursive feature elimination to select the top 2 features for tip amount. What are they?
    -Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

In [18]:
tips = sns.load_dataset('tips')

In [19]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [20]:
#new column total bill divided by size
tips['price_per_person'] = tips['total_bill']/tips['size']

In [21]:
tips.price_per_person=tips.price_per_person.round(2)

In [22]:
tips

,total_bill,tip,sex,smoker,day,time,size,price_per_person
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00
3,23.68,3.31,Male,No,Sun,Dinner,2,11.84
4,24.59,3.61,Female,No,Sun,Dinner,4,6.15
...,...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3,9.68
240,27.18,2.00,Female,Yes,Sat,Dinner,2,13.59
241,22.67,2.00,Male,Yes,Sat,Dinner,2,11.34
242,17.82,1.75,Male,No,Sat,Dinner,2,8.91


In [23]:
# -Before using any of the methods discussed in the lesson, 
# which features do you think would be most important for predicting the tip amount?
# I think total_bill, size, day/time

In [30]:
#data needs to be cleaned 
# Convert binary categorical variables to numeric
    tips['sex'] = tips.sex.map({'Female': 1, 'Male': 0})
    tips['smoker'] = tips.smoker.map({'Yes': 1, 'No': 0})
    
    
    # Get dummies for non-binary categorical variables
    dummy_df = pd.get_dummies(tips[['day', \
                              'time']], dummy_na=False, \
                              drop_first=True)
    
    # Concatenate dummy dataframe to original 
    tips = pd.concat([tips, dummy_df], axis=1)

In [31]:
tips.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   total_bill        244 non-null    float64 
 1   tip               244 non-null    float64 
 2   sex               244 non-null    category
 3   smoker            244 non-null    category
 4   day               244 non-null    category
 5   time              244 non-null    category
 6   size              244 non-null    int64   
 7   price_per_person  244 non-null    float64 
 8   day_Fri           244 non-null    uint8   
 9   day_Sat           244 non-null    uint8   
 10  day_Sun           244 non-null    uint8   
 11  time_Dinner       244 non-null    uint8   
dtypes: category(4), float64(3), int64(1), uint8(4)
memory usage: 10.2 KB


In [38]:
#clean data, change type to numeric so it can be modeled
tips.sex = pd.to_numeric(tips.sex, errors='coerce').astype('float64')
tips.smoker = pd.to_numeric(tips.smoker, errors='coerce').astype('float64')

In [37]:
tips.columns.tolist()

['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'price_per_person',
 'day_Fri',
 'day_Sat',
 'day_Sun',
 'time_Dinner']

In [24]:
# Use select k best to select the top 2 features for predicting tip amount. What are they?

In [33]:
train, validate, test = prepare_telco.split(tips, stratify_by=None)

In [34]:
train.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136 entries, 18 to 166
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   total_bill        136 non-null    float64 
 1   tip               136 non-null    float64 
 2   sex               136 non-null    category
 3   smoker            136 non-null    category
 4   day               136 non-null    category
 5   time              136 non-null    category
 6   size              136 non-null    int64   
 7   price_per_person  136 non-null    float64 
 8   day_Fri           136 non-null    uint8   
 9   day_Sat           136 non-null    uint8   
 10  day_Sun           136 non-null    uint8   
 11  time_Dinner       136 non-null    uint8   
dtypes: category(4), float64(3), int64(1), uint8(4)
memory usage: 6.9 KB


In [39]:
#remove target from X_train, remove categoricals so it can be modeled
X_train = train.drop(columns=['tip','day','time'])
y_train = train.tip

In [46]:
# from sklearn.feature_selection import SelectKBest, f_regression

# parameters: f_regression stats test, give me 2 features
f_selector = SelectKBest(f_regression, k=2)

# find the top 8 X's correlated with y
f_selector.fit(X_train, y_train)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature = X_train.iloc[:,feature_mask].columns.tolist()

In [47]:
f_feature

['total_bill', 'size']

In [ ]:
# Use recursive feature elimination to select the top 2 features for tip amount. What are they?

In [58]:
# from sklearn.linear_model import LinearRegression
# from sklearn.feature_selection import RFE

# initialize the ML algorithm
lm = LinearRegression()

# create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
rfe = RFE(lm, n_features_to_select=2)

# fit the data using RFE
rfe.fit(X_train,y_train)  

# get the mask of the columns selected
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = X_train.iloc[:,feature_mask].columns.tolist()

In [59]:
rfe_feature

['sex', 'size']

In [55]:
# Why do you think select k best and recursive feature elimination 
# might give different answers for the top features? Does this 
# change as you change the number of features your are selecting?
#rfe models combinations of the features, kbest tests each individual feature with target

#### 2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [60]:
def select_kbest(X_train, y_train, k):
    # parameters: f_regression stats test, give me 2 features
    f_selector = SelectKBest(f_regression, k=2)

# find the top 8 X's correlated with y
    f_selector.fit(X_train, y_train)

# boolean mask of whether the column was selected or not. 
    feature_mask = f_selector.get_support()

# get list of top K features. 
    f_feature = X_train.iloc[:,feature_mask].columns.tolist()
    return f_feature
    

#### 3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [61]:
def rfe (X_train, y_train, k):
    # initialize the ML algorithm
    lm = LinearRegression()

# create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
    rfe = RFE(lm, n_features_to_select=2)

# fit the data using RFE
    rfe.fit(X_train,y_train)  

# get the mask of the columns selected
    feature_mask = rfe.support_

# get list of the column names. 
    rfe_feature = X_train.iloc[:,feature_mask].columns.tolist()
    return rfe_feature
    

#### 4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out)

In [62]:
swiss =data('swiss')
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [ ]:
#drop target from xtrain, establish ytain
X_train = train.drop(columns=['tip','day','time'])
y_train = train.tip

In [ ]:
#minmax scaling

In [ ]:
#kbest function

In [ ]:
#rfe function